In [1]:
import tensorflow as tf
import pandas as pd
import random

from zipfile import ZipFile

In [2]:
random.seed(42)

train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")

def addextension(nm):
    return nm+".jpg"

train_df["Image_ID"] = train_df["Image_ID"].apply(addextension)
test_df["Image_ID"] = test_df["Image_ID"].apply(addextension)

print(train_df.head())

          Image_ID            class   xmin   ymin  width  height
0  ID_007FAIEI.jpg  fruit_woodiness   87.0   87.5  228.0   311.0
1  ID_00G8K1V3.jpg  fruit_brownspot   97.5   17.5  245.0   354.5
2  ID_00WROUT9.jpg  fruit_brownspot  156.5  209.5  248.0   302.5
3  ID_00ZJEEK3.jpg    fruit_healthy  125.0  193.0  254.5   217.0
4  ID_018UIENR.jpg  fruit_brownspot   79.5  232.5  233.5   182.0


In [3]:
def unzip(nm):
    with ZipFile(nm,"r") as zip:
        zip.extractall()

unzip("Train_Images.zip")
unzip("Test_Images.zip")

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    horizontal_flip = True,
    rotation_range = 10,
    zoom_range = 0.1,
    validation_split = 0.2,
)

train_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "training",
)

val_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation",
)

Found 3125 validated image filenames belonging to 3 classes.
Found 781 validated image filenames belonging to 3 classes.


In [5]:
from tensorflow.keras import layers
from tensorflow.keras import Model, Input

def model(input):
    x = layers.Rescaling(1./255)(input)
    x = layers.Conv2D(64,3,activation="relu",padding="same",strides=(2,2))(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(128,3,activation="relu",padding="same",strides=(2,2))(x)
    x = layers.Conv2D(128,3,activation="relu",padding="same",strides=(2,2))(x)
    x = layers.Conv2D(256,3,activation="relu",padding="same",strides=(2,2))(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(512,activation="relu")(x)
    x = layers.Dropout(0.2,seed=42)(x)
    x = layers.Dense(512,activation="relu")(x)
    x = layers.Dropout(0.2,seed=42)(x)
    output = layers.Dense(3,activation="softmax")(x)
    model = Model(input,output)
    return model

model = model(Input(shape=(256,256,3)))
model.summary()
model.compile(tf.keras.optimizers.RMSprop(),tf.keras.losses.CategoricalCrossentropy(),metrics=["accuracy"])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 256)         295168

In [6]:
if __name__=="__main__":
    checkpoint = tf.keras.callbacks.ModelCheckpoint("makerere.h5",save_weights_only=False,save_best_only=True,monitor="val_accuracy")
    model.fit(train_ds,epochs=20,validation_data=val_ds,callbacks=[checkpoint])
    best = tf.keras.models.load_model("makerere.h5")
    val_loss,val_acc = best.evaluate(val_ds)
    print("\nVal Accuracy: {:.2f} %".format(100*val_acc))
    print("Val Loss: {:.2f} %".format(100*val_loss))

Epoch 1/20
98/98 [==============================] - 48s 458ms/step - loss: 0.9913 - accuracy: 0.5309 - val_loss: 0.7422 - val_accuracy: 0.7157
Epoch 2/20
98/98 [==============================] - 40s 407ms/step - loss: 0.7625 - accuracy: 0.6781 - val_loss: 0.6664 - val_accuracy: 0.7042
Epoch 3/20
98/98 [==============================] - 40s 411ms/step - loss: 0.6386 - accuracy: 0.7181 - val_loss: 0.4853 - val_accuracy: 0.7951
Epoch 4/20
98/98 [==============================] - 41s 420ms/step - loss: 0.5645 - accuracy: 0.7578 - val_loss: 0.4733 - val_accuracy: 0.7926
Epoch 5/20
98/98 [==============================] - 43s 437ms/step - loss: 0.4959 - accuracy: 0.7907 - val_loss: 0.4742 - val_accuracy: 0.8015
Epoch 6/20
98/98 [==============================] - 44s 453ms/step - loss: 0.4258 - accuracy: 0.8275 - val_loss: 0.5075 - val_accuracy: 0.7798
Epoch 7/20
98/98 [==============================] - 41s 419ms/step - loss: 0.3913 - accuracy: 0.8483 - val_loss: 0.3381 - val_accuracy: 0.8720